In [ ]:
# Checking python version
!python --version

Python 3.10.12


In [ ]:
# Download and install Vieira
# The default python version is for 3.10, you may change the link according to your python versions.
%%capture
!wget https://github.com/vieira-artifact/vieira-artifact-aaai24/releases/download/v0.2.2/vieira-0.2.2-cp310-cp310-manylinux_2_31_x86_64.whl
!wget https://github.com/vieira-artifact/vieira-artifact-aaai24/releases/download/v0.2.2/vieira_ext-0.2.2-py3-none-any.whl
!wget https://github.com/vieira-artifact/vieira-artifact-aaai24/releases/download/v0.2.2/vieira_gpu-0.0.1-py3-none-any.whl
!wget https://github.com/vieira-artifact/vieira-artifact-aaai24/releases/download/v0.2.2/vieira_gpt-0.0.1-py3-none-any.whl
!pip install vieira-0.2.2-cp310-cp310-manylinux_2_31_x86_64.whl
!pip install vieira_ext-0.2.2-py3-none-any.whl
!pip install vieira_gpu-0.0.1-py3-none-any.whl
!pip install vieira_gpt-0.0.1-py3-none-any.whl

In [ ]:
# Import Vieira and related plugins
import vieira
import vieira_ext

# Vieira with Embeddings and GPT

In [ ]:
!wget https://raw.githubusercontent.com/vieira-artifact/vieira-artifact-aaai24/main/res/hotpot_examples.json

--2023-11-06 01:57:52--  https://raw.githubusercontent.com/vieira-artifact/vieira-artifact-aaai24/main/res/hotpot_examples.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38394 (37K) [text/plain]
Saving to: ‘hotpot_examples.json’

hotpot_examples.jso 100%[===================>]  37.49K  --.-KB/s    in 0.01s   

2023-11-06 01:57:53 (3.05 MB/s) - ‘hotpot_examples.json’ saved [38394/38394]



In [ ]:
import json
dataset = json.load(open('hotpot_examples.json', 'r'))

In [ ]:
# Add your OpenAI API key if you want to run the example
import os
os.environ['OPENAI_API_KEY'] = "YOUR-OPENAI-API-KEY-HERE"

In [ ]:
# Configure Vieira plugins
import argparse
plugins = vieira_ext.PluginRegistry()

parser = argparse.ArgumentParser()
plugins.setup_argument_parser(parser)
known_args, unknown_args = parser.parse_known_args()
plugins.configure(known_args, unknown_args)

In [ ]:
PROGRAM = """
@gpt_encoder
type $embed_text(String) -> Tensor

type question(q: String)

type context(id: i32, c: String)

rel relevant(id) = id := top<2>(id: question(q) and context(id, c) and soft_eq<Tensor>($embed_text(q), $embed_text(c)))
rel relevant_context($string_concat(c1, "\n", c2)) = relevant(id1) and relevant(id2) and id1 < id2 and context(id1, c1) and context(id2, c2)

@gpt(
  prompt="Given {{context}}\n{{question}}\nPlease think step-by-step {{answer}}",
  model="gpt-4",
  debug=false,
)
type qa(bound question: String, bound context: String, answer: String)

rel answer(a) = question(q) and relevant_context(c) and qa(q, c, a)

query answer
"""

In [ ]:
# Hotpot QA
for idx, example in enumerate(dataset):
  ctx = vieira.Context()

  plugins.load_into_ctx(ctx)
  ctx.add_program(PROGRAM)
  ctx.set_non_probabilistic(['question', 'context'])

  question = example['question']
  ctx.add_facts('question', [(question,)])
  for i, document in enumerate(example['context']):
    ctx.add_facts('context', [(i, document,)])

  ctx.run()

  result = list(ctx.relation('answer'))[0][0]
  gt_answer = example['answer']
  print(f"{idx + 1}: {question}\nmodel_answer: {result}\ncorrect answer: {gt_answer}\n\n")

1: Were Scott Derrickson and Ed Wood of the same nationality?
model_answer: Yes, Scott Derrickson and Ed Wood are of the same nationality
correct answer: yes


2: The arena where the Lewiston Maineiacs played their home games can seat how many people?
model_answer: The information provided does not include the seating capacity of the arena where the Lewiston Maineiacs played their home games.
correct answer: 3,677 seated


3: Are Local H and For Against both from the United States?
model_answer: 
correct answer: yes


4: When did the English local newspaper, featuring the sculpture and war memorial in the Forbury gardens, change names?
model_answer: The information provided does not contain details about when the English local newspaper, featuring the sculpture and war memorial in the Forbury gardens, changed names.
correct answer: 2009


5: What airport serviced by American Airlines Shuttle, is also the largest airport in the New England region and 17th busiest in the U.S?
model_answe